<img align="Left" alt="Dados_ao_Cubo" src="../shared_content/DADOS-AO-CUBO-Versão-04-1.png" />
<img align="right" alt="MLNet" src="../shared_content/EJ3KsfJWsAEtDaZ.png" />

## Davi Ramos -> Cientista de Dados 👋
(davi.info@gmail.com)

[![Linkedin Badge](https://img.shields.io/badge/-LinkedIn-blue?style=flat-square&logo=Linkedin&logoColor=white&link=https://www.linkedin.com/in/davi-ramos/)](https://www.linkedin.com/in/davi-ramos/)
[![Twitter Badge](https://img.shields.io/badge/-Twitter-1DA1F2?style=flat-square&logo=Twitter&logoColor=white&link=https://twitter.com/Daviinfo/)](https://twitter.com/Daviinfo/)
<a href="https://github.com/DaviRamos"><img src="https://img.shields.io/github/followers/DaviRamos.svg?label=GitHub&style=social" alt="GitHub"></a>

# ML.Net - Regressão Linear - 2ª Versão 

## Este exemplo irá criar um modelo de regressão linear para prever os preços das residências utilizando os seus dados de tamanho e Preço.

In [174]:
// Instalar os Pacotes do Nuget

// ML.NET
#r "nuget:Microsoft.ML"  

// ML.NET FastTree 
#r "nuget:Microsoft.ML.FastTree" 
   
//  DataFrame
#r "nuget:Microsoft.Data.Analysis"

//  XPlot
#r "nuget:XPlot.Plotly,2.0.0"

In [175]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using Microsoft.ML.Trainers.FastTree;
using System.IO;
using System.IO.Compression;
using System.Net.Http;
using System.Globalization;
using XPlot.Plotly;

using static Microsoft.ML.TrainCatalogBase;
using static Microsoft.ML.DataOperationsCatalog;

In [176]:
// Caminho dos arquivos
string housingPath = "./Datasets/Housing/housing.csv";
string modelPath = @"./HouseModelv2.zip";

In [177]:
//Download do Dataset
if (!File.Exists(housingPath))
{
    var contents = new HttpClient()
        .GetStringAsync("https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv").Result;
        
    File.WriteAllText(housingPath, contents);
}

In [178]:
// Formatador do Dataframe
Formatter.Register<DataFrame>((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 10;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");

## Análise Exploratória de Dados (EDA)

In [179]:
var df = DataFrame.LoadCsv(housingPath);

In [180]:
df.Head(5)

0,-122.23,37.88,41,880,129,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280,565,259,3.8462,342200,NEAR BAY


In [181]:
df.Sample(5)

0,-122.02,37.01,20,1005,138,345,129,10.0968,500001,NEAR OCEAN
1,-122.15,37.41,29,473,103,359,87,7.0309,475000,NEAR BAY
2,-121.7,39.36,37,2330,495,1505,470,2.0474,56000,INLAND
3,-122.13,37.15,39,2854,613,1338,518,3.9423,180300,NEAR OCEAN
4,-121.62,37.09,37,1593,303,1030,287,3.9306,260700,<1H OCEAN


In [182]:
df.Tail(5)

0,-121.09,39.48,25,1665,374,845,330,1.5603,78100,INLAND
1,-121.21,39.49,18,697,150,356,114,2.5568,77100,INLAND
2,-121.22,39.43,17,2254,485,1007,433,1.7,92300,INLAND
3,-121.32,39.43,18,1860,409,741,349,1.8672,84700,INLAND
4,-121.24,39.37,16,2785,616,1387,530,2.3886,89400,INLAND


In [183]:
df.AddPrefix("HOUSE_")

0,-122.23,37.88,41,880,129,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280,565,259,3.8462,342200,NEAR BAY
5,-122.25,37.85,52,919,213,413,193,4.0368,269700,NEAR BAY
6,-122.25,37.84,52,2535,489,1094,514,3.6591,299200,NEAR BAY
7,-122.25,37.84,52,3104,687,1157,647,3.12,241400,NEAR BAY
8,-122.26,37.84,42,2555,665,1206,595,2.0804,226700,NEAR BAY
9,-122.25,37.84,52,3549,707,1551,714,3.6912,261100,NEAR BAY


In [184]:
df.AddSuffix("_data")

0,-122.23,37.88,41,880,129,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280,565,259,3.8462,342200,NEAR BAY
5,-122.25,37.85,52,919,213,413,193,4.0368,269700,NEAR BAY
6,-122.25,37.84,52,2535,489,1094,514,3.6591,299200,NEAR BAY
7,-122.25,37.84,52,3104,687,1157,647,3.12,241400,NEAR BAY
8,-122.26,37.84,42,2555,665,1206,595,2.0804,226700,NEAR BAY
9,-122.25,37.84,52,3549,707,1551,714,3.6912,261100,NEAR BAY


In [185]:
df.Description()

0,Length (excluding null values),20640,20640,20640,20640,20433,20640,20640,20640,20640
1,Max,-114.31,41.95,52,39320,6445,35682,6082,15.0001,500001
2,Min,-124.35,32.54,1,2,0,3,1,0.4999,14999
3,Mean,-119.569115,35.631866,28.639486,2635.7588,532.4762,1425.4779,499.53967,3.8706622,206854.97


In [186]:
df.Info()

0,DataType,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single,System.String
1,Length (excluding null values),20640,20640,20640,20640,20433,20640,20640,20640,20640,20640


In [187]:
df.DropNulls()

0,-122.23,37.88,41,880,129,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280,565,259,3.8462,342200,NEAR BAY
5,-122.25,37.85,52,919,213,413,193,4.0368,269700,NEAR BAY
6,-122.25,37.84,52,2535,489,1094,514,3.6591,299200,NEAR BAY
7,-122.25,37.84,52,3104,687,1157,647,3.12,241400,NEAR BAY
8,-122.26,37.84,42,2555,665,1206,595,2.0804,226700,NEAR BAY
9,-122.25,37.84,52,3549,707,1551,714,3.6912,261100,NEAR BAY


In [188]:
df = df.DropNulls()

0,-122.23,37.88,41,880,129,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280,565,259,3.8462,342200,NEAR BAY
5,-122.25,37.85,52,919,213,413,193,4.0368,269700,NEAR BAY
6,-122.25,37.84,52,2535,489,1094,514,3.6591,299200,NEAR BAY
7,-122.25,37.84,52,3104,687,1157,647,3.12,241400,NEAR BAY
8,-122.26,37.84,42,2555,665,1206,595,2.0804,226700,NEAR BAY
9,-122.25,37.84,52,3549,707,1551,714,3.6912,261100,NEAR BAY


In [189]:
var bedroomsMean = df.Columns["total_bedrooms"].Mean();

df.FillNulls(bedroomsMean.ToString(), inPlace: true);
df

0,-122.23,37.88,41,880,129,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280,565,259,3.8462,342200,NEAR BAY
5,-122.25,37.85,52,919,213,413,193,4.0368,269700,NEAR BAY
6,-122.25,37.84,52,2535,489,1094,514,3.6591,299200,NEAR BAY
7,-122.25,37.84,52,3104,687,1157,647,3.12,241400,NEAR BAY
8,-122.26,37.84,42,2555,665,1206,595,2.0804,226700,NEAR BAY
9,-122.25,37.84,52,3549,707,1551,714,3.6912,261100,NEAR BAY


In [190]:
var filtered = df[df.Columns["median_house_value"].ElementwiseGreaterThan(500000)];
filtered

0,-122.27,37.8,52,249,78,396,85,1.2434,500001,NEAR BAY
1,-122.25,37.87,52,609,236,1349,250,1.1696,500001,NEAR BAY
2,-122.24,37.86,52,1668,225,517,214,7.8521,500001,NEAR BAY
3,-122.24,37.85,52,3726,474,1366,496,9.3959,500001,NEAR BAY
4,-122.23,37.83,52,2990,379,947,361,7.8772,500001,NEAR BAY
5,-122.22,37.82,39,2492,310,808,315,11.8603,500001,NEAR BAY
6,-122.22,37.82,42,2991,335,1018,335,13.499,500001,NEAR BAY
7,-122.23,37.82,52,3242,366,1001,352,12.2138,500001,NEAR BAY
8,-122.23,37.82,52,3494,396,1192,383,12.3804,500001,NEAR BAY
9,-122.23,37.82,52,1611,203,556,179,8.7477,500001,NEAR BAY


In [191]:
var newDf = df.Clone();
newDf

0,-122.23,37.88,41,880,129,322,126,8.3252,452600,NEAR BAY
1,-122.22,37.86,21,7099,1106,2401,1138,8.3014,358500,NEAR BAY
2,-122.24,37.85,52,1467,190,496,177,7.2574,352100,NEAR BAY
3,-122.25,37.85,52,1274,235,558,219,5.6431,341300,NEAR BAY
4,-122.25,37.85,52,1627,280,565,259,3.8462,342200,NEAR BAY
5,-122.25,37.85,52,919,213,413,193,4.0368,269700,NEAR BAY
6,-122.25,37.84,52,2535,489,1094,514,3.6591,299200,NEAR BAY
7,-122.25,37.84,52,3104,687,1157,647,3.12,241400,NEAR BAY
8,-122.26,37.84,42,2555,665,1206,595,2.0804,226700,NEAR BAY
9,-122.25,37.84,52,3549,707,1551,714,3.6912,261100,NEAR BAY


In [192]:
Chart.Plot(
    new Graph.Histogram()
    {
        x = df.Columns["median_house_value"],
        nbinsx = 20
    }
)

In [193]:
Chart.Plot(
    new Graph.Histogram()
    {
         x = df.GroupBy("ocean_proximity").Count(),
        y = df.Columns["ocean_proximity"]
    }
)

In [194]:
Chart.Plot(
    new Graph.Box()
    {
         y = df.Columns[8],
        name = df.Columns[8].Name
    }
)

In [195]:
var chart2 = Chart.Plot(new List<Graph.Box> 
{ 
    new Graph.Box()
    {
        y = df.Columns[0],
        name = df.Columns[0].Name
    },
    new Graph.Box()
    {
        y = df.Columns[1],
        name = df.Columns[1].Name
    },
    new Graph.Box()
    {
        y = df.Columns[2],
        name = df.Columns[2].Name
    },
    new Graph.Box()
    {
        y = df.Columns[3],
        name = df.Columns[3].Name
    },
    new Graph.Box()
    {
        y = df.Columns[5],
        name = df.Columns[4].Name
    },  
    new Graph.Box()
    {
        y = df.Columns[5],
        name = df.Columns[5].Name
    },
    new Graph.Box()
    {
        y = df.Columns[6],
        name = df.Columns[6].Name
    },
    new Graph.Box()
    {
        y = df.Columns[7],
        name = df.Columns[7].Name
    },
    new Graph.Box()
    {
        y = df.Columns[9],
        name = df.Columns[9].Name
    }
});

var layout2 = new Layout.Layout(){ title="EDA - House Price", showlegend = false };
chart2.WithLayout(layout2);

display(chart2);

In [196]:
public class ModelInput
{
    [LoadColumn(0)]
    public float Longitude { get; set; }
    [LoadColumn(1)]
    public float Latitude { get; set; }
    [LoadColumn(2)]
    public float Housing_median_age { get; set; }
    [LoadColumn(3)]
    public float Total_rooms { get; set; }
    [LoadColumn(4)]
    public float Total_bedrooms { get; set; }
    [LoadColumn(5)]
    public float Population { get; set; }
    [LoadColumn(6)]
    public float Households { get; set; }
    [LoadColumn(7)]
    public float Median_income { get; set; }
    [ColumnName("Label"), LoadColumn(8)]
    public float Median_house_value { get; set; }
    [LoadColumn(9)]
    public string Ocean_proximity { get; set; }
}

public class ModelOutput
{
    [ColumnName("Score")]
    public float Score { get; set; }
}

In [197]:
// Criar o MLContext para ser compartilhado entre os objetos de fluxo de trabalho de criação do modelo
// Definir uma semente aleatória para resultados repetíveis/determinísticos nos vários treinamentos.
MLContext mlContext = new MLContext(seed: 1);

In [198]:
// Métricas de Avalição do Modelo
public static void PrintRegressionMetrics(RegressionMetrics metrics)
{
    Console.WriteLine($"*************************************************");
    Console.WriteLine($"*       Metrics for Regression model      ");
    Console.WriteLine($"*------------------------------------------------");
    Console.WriteLine($"*       LossFn:        {metrics.LossFunction:0.##}");
    Console.WriteLine($"*       R2 Score:      {metrics.RSquared:0.##}");
    Console.WriteLine($"*       Absolute loss: {metrics.MeanAbsoluteError:#.##}");
    Console.WriteLine($"*       Squared loss:  {metrics.MeanSquaredError:#.##}");
    Console.WriteLine($"*       RMS loss:      {metrics.RootMeanSquaredError:#.##}");
    Console.WriteLine($"*       RMS loss:      {metrics.RootMeanSquaredError:#.##}");
    Console.WriteLine($"*************************************************");
}

public static void PrintRegressionFoldsAverageMetrics(IEnumerable<TrainCatalogBase.CrossValidationResult<RegressionMetrics>> crossValidationResults)
{
    var L1 = crossValidationResults.Select(r => r.Metrics.MeanAbsoluteError);
    var L2 = crossValidationResults.Select(r => r.Metrics.MeanSquaredError);
    var RMS = crossValidationResults.Select(r => r.Metrics.RootMeanSquaredError);
    var lossFunction = crossValidationResults.Select(r => r.Metrics.LossFunction);
    var R2 = crossValidationResults.Select(r => r.Metrics.RSquared);
    
    Console.WriteLine($"*************************************************************************************************************");
    Console.WriteLine($"*       Metrics for Regression model      ");
    Console.WriteLine($"*------------------------------------------------------------------------------------------------------------");
    Console.WriteLine($"*       Average L1 Loss:       {L1.Average():0.###} ");
    Console.WriteLine($"*       Average L2 Loss:       {L2.Average():0.###}  ");
    Console.WriteLine($"*       Average RMS:           {RMS.Average():0.###}  ");
    Console.WriteLine($"*       Average Loss Function: {lossFunction.Average():0.###}  ");
    Console.WriteLine($"*       Average R-squared:     {R2.Average():0.###}  ");    
    Console.WriteLine($"*************************************************************************************************************");
}

In [199]:
private static IDataView trainingDataView;
private static IDataView testingDataView;

//Configuração do carregamento de dados         
IDataView fullData = mlContext.Data.LoadFromTextFile<ModelInput>(path: housingPath, hasHeader: true, 
                                                                         separatorChar: ',', allowQuoting: true, allowSparse: false);

// Divide o conjunto de dados em duas partes: TrainingDataset (80%) e TestDataset (20%)
DataOperationsCatalog.TrainTestData trainTestData = mlContext.Data.TrainTestSplit(fullData, testFraction: 0.2);
trainingDataView = trainTestData.TrainSet;
testingDataView = trainTestData.TestSet;

In [229]:
// Criar e treinar o modelo
// Configuração do processo de Pipeline de transformação de dados 
var dataProcessPipeline = mlContext.Transforms.Categorical.OneHotEncoding(new[] 
                                                { new InputOutputColumnPair("Ocean_proximity", "Ocean_proximity") })
                                                  .Append(mlContext.Transforms.Concatenate("Features", new[] { "Ocean_proximity", 
                                                                                        "Longitude", "Latitude", "Housing_median_age", 
                                                                                        "Total_rooms", "Total_bedrooms", "Population", 
                                                                                                        "Households", "Median_income" })
                                                 .AppendCacheCheckpoint(mlContext));
// Definir o algoritmo de treinamento
var trainer = mlContext.Regression.Trainers.Sdca(labelColumnName: "Label", featureColumnName: "Features");

var trainingPipeline = dataProcessPipeline.Append(trainer);


(3,27): warning CS1701: Assuming assembly reference 'Microsoft.AspNetCore.Html.Abstractions, Version=2.2.0.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' used by 'Microsoft.DotNet.Interactive.Formatting' matches identity 'Microsoft.AspNetCore.Html.Abstractions, Version=3.1.7.0, Culture=neutral, PublicKeyToken=adb9793829ddae60' of 'Microsoft.AspNetCore.Html.Abstractions', you may need to supply runtime policy



In [230]:
Console.WriteLine("=============== Treinando o modelo ===============");

var model = trainingPipeline.Fit(trainingDataView);

Console.WriteLine("=============== Fim do Processo de Treinamento ===============");

=============== Treinando o modelo ===============
=============== Fim do Processo de Treinamento ===============


In [231]:
// Avalie a precisão do modelo
IDataView predictions = model.Transform(testingDataView);
Console.WriteLine("===== Avaliando a acurácia do modelo com dados de teste =====");
var metrics = mlContext.Regression.Evaluate(data:predictions, labelColumnName:"Label", scoreColumnName: "Score"); 
PrintRegressionMetrics(metrics);

===== Avaliando a acurácia do modelo com dados de teste =====
*************************************************
*       Metrics for Regression model      
*------------------------------------------------
*       LossFn:        9940724537.56
*       R2 Score:      0.26
*       Absolute loss: 71203.82
*       Squared loss:  9940724573.98
*       RMS loss:      99703.18
*       RMS loss:      99703.18
*************************************************


In [203]:
// Salvar/persistir o modelo treinado em um arquivo .ZIP
Console.WriteLine($"=============== Salvando o modelo  ===============");
mlContext.Model.Save(model, trainingDataView.Schema, modelPath); 
Console.WriteLine("O Modelo foi salvo em {0}", modelPath);

=============== Salvando o modelo  ===============
O Modelo foi salvo em ./HouseModelv2.zip


In [204]:
Console.WriteLine("=============== Previsão para um único caso  ===============");

ModelInput sampleData = new ModelInput()
{
    Longitude = -122.23F,
    Latitude = 37.88F,
    Housing_median_age = 41F,
    Total_rooms = 880F,
    Total_bedrooms = 129F,
    Population = 322F,
    Households = 126F,
    Median_income = 8.3252F,
    Ocean_proximity = @"NEAR BAY",
};

=============== Previsão para um único caso  ===============


In [222]:
ITransformer model = mlContext.Model.Load(modelPath, out var modelInputSchema);
// Criar um mecanismo de predição relacionado ao modelo treinado carregado
var predEngine = mlContext.Model.CreatePredictionEngine <ModelInput, ModelOutput>(model);

//Score
var resultprediction = predEngine.Predict (sampleData);

Console.WriteLine("Usando o modelo para fazer predição única - Comparando o valor atual de Median_house_value com o valor previsto de para os dados de amostra de Median_house_value ... \n \n");
Console.WriteLine($"Longitude: {sampleData.Longitude}");
Console.WriteLine($"Latitude: {sampleData.Latitude}");
Console.WriteLine($"Housing_median_age: {sampleData.Housing_median_age}");
Console.WriteLine($"Total_rooms: {sampleData.Total_rooms}");
Console.WriteLine($"Total_bedrooms: {sampleData.Total_bedrooms}");
Console.WriteLine($"Population: {sampleData.Population}");
Console.WriteLine($"Households: {sampleData.Households}");
Console.WriteLine($"Median_income: {sampleData.Median_income}");
Console.WriteLine($"Ocean_proximity: {sampleData.Ocean_proximity}");
Console.WriteLine ($"\n \nO Preço previsto para este tamanho é: {resultprediction.Score} \n \n");
Console.WriteLine ("=============== Fim do processo ===============");

Usando o modelo para fazer predição única - Comparando o valor atual de Median_house_value com o valor previsto de para os dados de amostra de Median_house_value ... 
 

Longitude: -122.23
Latitude: 37.88
Housing_median_age: 41
Total_rooms: 880
Total_bedrooms: 129
Population: 322
Households: 126
Median_income: 8.3252
Ocean_proximity: NEAR BAY

 
O Preço previsto para este tamanho é: 295181.94 
 

=============== Fim do processo ===============


## Fluxo de trabalho de código

Neste Artigo  percorremos todas as etapas abaixo
<ul>
   <li>Coletar e carregar dados de treinamento em um objeto IDataView</li>
   <li>Especifique um pipeline de operações para extrair recursos e aplicar um algoritmo de aprendizado de máquina</li>
   <li>Treinar um modelo chamando Fit() no pipeline</li>
   <li>Avaliar o modelo e iterar para melhorar</li>
   <li>Salvar o modelo em um formato binário para uso em um aplicativo</li>
   <li>Carregar o modelo de volta para um objeto ITransformer</li>
   <li>Fazer previsões chamando CreatePredictionEngine.Predict()</li>
</ul>

![Fluxo](../shared_content/mldotnet-annotated-workflow.png)

# Em nosso próximo artigo veremos como implantar o modelo criado numa api Web para consumo por outra aplicações.